<a href="https://colab.research.google.com/github/catalinamagalvan/PySimMIBCI/blob/main/notebooks/Example_generate_data_different_user_capabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Simulation of users with different brain modulation capabilities
This notebook illustrates how to generate MI-BCI data related to users with different capabilities to control a BCI.
For doing so, two strategies are presented: 

1. The first strategy employs different percentages of desynchronization in the $\alpha$ band to simulate the typical ERD in the contralateral hand motor area. In this example, subject S20 is simulated, considering a 20% of desynchronization. In the simulation of S20, all the trials have the same modulation level.
2. The second strategy consists in the inclusion of a certain proportion of failed MI trials, that is, trials without ERD in the corresponding area. In this example, subject SF30 is simulated, considering a 30% of trials without modulation. For SF30, the ideal percentage of desynchronization (50%) is used.

Notes:

* These two strategies can also be combined to artificially generate users with different capabilities related to the control of a MI-BCI.
* In order to see the classification results obtained for S20 see the example [Example_cross_validation_simdataset.ipynb](https://github.com/catalinamagalvan/PySimMIBCI/blob/main/FBCNet%20Toolbox/Example_cross_validation_simdataset.ipynb).




##Clone GitHub repository

In [1]:
!git clone https://github.com/catalinamagalvan/PySimMIBCI.git

Cloning into 'PySimMIBCI'...
remote: Enumerating objects: 564, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 564 (delta 146), reused 100 (delta 60), pack-reused 322
Receiving objects: 100% (564/564), 2.00 MiB | 10.37 MiB/s, done.
Resolving deltas: 100% (298/298), done.


##Install and import the required Python libraries

In [2]:
!pip install mne==1.3.0
!pip install fooof==1.0.0
!pip install colorednoise==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 KB 3.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from scipy.io import loadmat
import os
import mne
import numpy as np
from mne.datasets import fetch_fsaverage
from mne.decoding import CSP
import sys
sys.path.insert(1, os.path.join('PySimMIBCI', 'src'))
from simulation import (set_up_source_forward, generate_when, generate_what,
                        generate_what_failed, generate_where,
                        save_mat_simulated_data)
from noise import add_aperiodic_activity, add_eye_movement
sys.path.insert(1, os.path.join('PySimMIBCI', 'utils'))
from datasets import (load_and_epoch_OpenBMI_by_ftp, create_BCIIV1_info,
                      raw_from_OpenBMI_data_by_ftp)
from plots import plot_raw_2_channels

## Set up general simulation parameters

In [4]:
# MI IDs
MI_tasks_IDs = {'MI/left': 1, 'MI/right': 2}
# Sample frequency
sfreq = 1000
# Number of trials
N_trials = 50
# MI trial duration in ms
MI_duration = 4500

Note: here, for both simulations user g from BCI competition IV dataset 1 is employed as the guide BCI-user. Thus, its aperiodic and aperiodic parameters are employed for the simulation.

In [5]:
user_params = {'aperiodic_params': [-7.17, 1.14],
               'peak_params': {'G_precentral-lh': [9.54, 1, 2.52],
                               'G_precentral-rh': [10.13, 0.8, 1.62]}}

Generate suitable mne Info object for BCI competition IV dataset 1 data.

In [6]:
info = create_BCIIV1_info()

#Simulation of subject S20
Parameters:
*   Percentage of ERD: 20%,
*   Percentage of failed trials: 0%.



In [7]:
reduction = 0.2

##Set up source space and compute forward solution.

Here, *fsaverage* head model is employed, which is a template brain based on a combination of 40 MRI scans of real brains.

In [8]:
subject = 'fsaverage'
fwd, source_simulator = set_up_source_forward(subject, info)

##Generate task-related when, what and where information
Create events, waveform and spatial information for the task-related component in right hand MI vs left hand MI scenario.

In [9]:
events_info = {0: {'label': 'MI/left', 'duration': MI_duration},
               1: {'label': 'MI/right', 'duration': MI_duration}}
when_S20 = generate_when(events_info, N_trials, sfreq)
what = generate_what(MI_tasks_IDs.keys(), when_S20, user_params, MI_duration,
                     sfreq, N_trials, reduction)
where = generate_where(subject)

##Add task-related component to source simulator object.

In [10]:
for label in what.keys():
    for task_ID, task in enumerate(MI_tasks_IDs.keys(), 1):
        when_task = when_S20[np.where(when_S20[:, 2] == task_ID)[0]]
        source_simulator.add_data(label=where[label],
                                  waveform=what[label][task],
                                  events=when_task)

##Simulate raw data

Project the source time series to sensor space.
For doing so, the `mne.simulation.SourceSimulator` object is passed directly to the `mne.simulation.simulate_raw` function.

**Note:** Running this cell takes about 5 min when 50 trials are generated. This duration is directly related to the number of simulated trials (variable *N_trials* at *Set up simulation parameters* cell).

In [11]:
raw_sim_S20 = mne.simulation.simulate_raw(info, source_simulator, forward=fwd)
raw_sim_S20.set_eeg_reference(ref_channels='average')

<RawArray | 41 x 225000 (225.0 s), ~70.4 MB, data loaded>

##Add non-task-related components


*   Aperiodic activity (background noise) to raw simulated data Here, user-specific aperiodic parameters are employed.
*   Eog artifacts by MNE function.
*   Eye-movement artifacts.



In [12]:
raw_sim_S20 = add_aperiodic_activity(raw_sim_S20,
                                     exponent=user_params['aperiodic_params'][1],
                                     offset=user_params['aperiodic_params'][0])
mne.simulation.add_eog(raw_sim_S20)
add_eye_movement(raw_sim_S20)

##Epoch data

In [13]:
epochs_sim_S20 = mne.Epochs(raw_sim_S20, when_S20, MI_tasks_IDs, tmin=0,
                            tmax=4, baseline=None)

##Save generated data (optional)
Save simulated data in a .mat file compatible with FBCNet Toolbox functions.

In [14]:
spath = os.path.join('PySimMIBCI', 'FBCNet Toolbox', 'data', 'simdataset')
save_mat_simulated_data(raw_sim_S20, when_S20, spath, 's20.mat')

#Generation of subject SF30
*   Percentage of ERD: 50% (ideal),
*   Percentage of failed trials: 30%.




In [15]:
reduction = 0.5

##Set up source space and compute forward solution.

Here, fsavarage head model is employed, which is a template brain based on a combination of 40 MRI scans of real brains.

In [16]:
subject = 'fsaverage'
fwd, source_simulator = set_up_source_forward(subject, info)

##Generate task-related when, what and where information
Events, waveform and spatial information for the task-related component in right hand MI vs left hand MI scenario.

In [17]:
events_info = {0: {'label': 'MI/left', 'duration': MI_duration},
               1: {'label': 'MI/right', 'duration': MI_duration}}
p_failed = 0.3
when_SF30 = generate_when(events_info, N_trials, sfreq)
what = generate_what_failed(MI_tasks_IDs.keys(), when_SF30, user_params, MI_duration, sfreq,
                            N_trials, reduction, p_failed)
where = generate_where(subject)

##Add task-related component to source simulator object

In [18]:
for label in what.keys():
    for task_ID, task in enumerate(MI_tasks_IDs.keys(), 1):
        when_task = when_SF30[np.where(when_SF30[:, 2] == task_ID)[0]]
        source_simulator.add_data(label=where[label],
                                  waveform=what[label][task],
                                  events=when_task)

##Simulate raw data. 

Project the source time series to sensor space.
For doing so, the `mne.simulation.SourceSimulator` object is given directly to the `mne.simulation.simulate_raw` function.


**Note:** Running this cell takes about 5 min when 50 trials are generated. This duration is directly related to the number of simulated trials (variable *N_trials* at *Set up simulation parameters* cell).

In [19]:
raw_sim_SF30 = mne.simulation.simulate_raw(info, source_simulator, forward=fwd)
raw_sim_SF30.set_eeg_reference(ref_channels='average')

<RawArray | 41 x 225000 (225.0 s), ~70.4 MB, data loaded>

##Add non-task-related components:

*   Aperiodic activity (background noise). Here, user-specific aperiodic parameters (exponent and offset) are employed.
*   Eog artifacts by [mne.simulation.add_eog](https://mne.tools/stable/generated/mne.simulation.add_eog.html) function.
*   Eye-movement artifacts.

In [20]:
raw_sim_SF30 = add_aperiodic_activity(raw_sim_SF30,
                                      exponent=user_params['aperiodic_params'][1],
                                      offset=user_params['aperiodic_params'][0])
mne.simulation.add_eog(raw_sim_SF30)
add_eye_movement(raw_sim_SF30)

##Epoch data

In [21]:
epochs_sim_SF30 = mne.Epochs(raw_sim_SF30, when_SF30, MI_tasks_IDs, tmin=0,
                             tmax=4, baseline=None)

##Save generated data (optional)
Save simulated data in a .mat file compatible with FBCNet Toolbox functions.

In [22]:
spath = os.path.join('PySimMIBCI', 'FBCNet Toolbox', 'data', 'simdataset')
save_mat_simulated_data(raw_sim_SF30, when_SF30, spath, 'sf30.mat')